In [1]:
import os
import platform
import random
import shutil

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor 
from torch.cuda.amp import autocast, GradScaler 
from torchvision.utils import draw_bounding_boxes, save_image

import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import torchvision.transforms.functional as F

import labelme2coco
from pycocotools.coco import COCO

import pandas as pd 
import warnings 
warnings.filterwarnings("ignore")

In [2]:
HOME_DIR = '/Users/cucum/Downloads/COS40007/week 5'
os.chdir(HOME_DIR) 
DATASETS_DIR = f'{HOME_DIR}'
DATA_DIR = f'{DATASETS_DIR}/train'
NO_RUST_DATASET_FOLDER = f'{DATA_DIR}/no rust'
RUST_DATASET_FOLDER = f'{DATA_DIR}/rust'
TEST_DIR = f'{DATASETS_DIR}/test'
LOG_LABEL_DATASET_FOLDER = f'{DATASETS_DIR}/log-labelled'
COCO_ANNOTATIONS_DIR = f'{DATASETS_DIR}/coco_annotations'

MY_LOG_LABEL_DATASET_FOLDER = f'{DATASETS_DIR}/log-labelled'
MY_COCO_ANNOTATIONS_DIR = f'{DATASETS_DIR}coco_annotations'

os.makedirs(TEST_DIR, exist_ok=True)
os.makedirs(f'{TEST_DIR}/no rust', exist_ok=True)
os.makedirs(f'{TEST_DIR}/rust', exist_ok=True)
os.makedirs(COCO_ANNOTATIONS_DIR, exist_ok=True)
os.makedirs(MY_COCO_ANNOTATIONS_DIR, exist_ok=True)

print(f'HOME_DIR: {HOME_DIR}')
print(f'DATASETS_DIR: {DATASETS_DIR}')
print(f'DATA_DIR: {DATA_DIR}') 
print(f'TEST_DIR: {TEST_DIR}')
print(f'RUST_DATASET_FOLDER: {RUST_DATASET_FOLDER}')
print(f'NO_RUST_DATASET_FOLDER: {NO_RUST_DATASET_FOLDER}')
print(f'COCO_ANNOTATIONS_DIR: {COCO_ANNOTATIONS_DIR}')

HOME_DIR: /Users/cucum/Downloads/COS40007/week 5
DATASETS_DIR: /Users/cucum/Downloads/COS40007/week 5
DATA_DIR: /Users/cucum/Downloads/COS40007/week 5/train
TEST_DIR: /Users/cucum/Downloads/COS40007/week 5/test
RUST_DATASET_FOLDER: /Users/cucum/Downloads/COS40007/week 5/train/rust
NO_RUST_DATASET_FOLDER: /Users/cucum/Downloads/COS40007/week 5/train/no rust
COCO_ANNOTATIONS_DIR: /Users/cucum/Downloads/COS40007/week 5/coco_annotations


In [3]:
# Set directory
labelme_folder = MY_LOG_LABEL_DATASET_FOLDER
export_dir = MY_COCO_ANNOTATIONS_DIR

# Convert LabelMe annotations to COCO format
category_id_start = 1  # Start category IDs from 1
labelme2coco.convert(labelme_folder, export_dir, category_id_start=category_id_start)

print("LabelMe annotations converted to COCO format for log detection.")

There are 600 listed files in folder /Users/cucum/Downloads/COS40007/week 5/log-labelled.


Converting labelme annotations to COCO format: 100%|██████████| 600/600 [00:02<00:00, 277.98it/s]
10/23/2024 18:52:47 - INFO - labelme2coco -   Converted annotations in COCO format is exported to \Users\cucum\Downloads\COS40007\week 5coco_annotations\dataset.json


LabelMe annotations converted to COCO format for log detection.


In [4]:
# Custom COCO dataset class
class COCODataset(Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.transforms = transforms

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        img_info = coco.loadImgs(img_id)[0]
        path = img_info['file_name']

        # Load image
        img_path = os.path.join(self.root, path)
        img = Image.open(img_path).convert('RGB')

        # Load annotations
        num_objs = len(anns)
        boxes = []
        labels = []
        masks = []

        for ann in anns:
            xmin = ann['bbox'][0]
            ymin = ann['bbox'][1]
            width = ann['bbox'][2]
            height = ann['bbox'][3]
            xmax = xmin + width
            ymax = ymin + height
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(ann['category_id'])
            masks.append(coco.annToMask(ann))

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        masks = torch.as_tensor(np.stack(masks, axis=0), dtype=torch.uint8)

        image_id = torch.tensor([img_id])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['masks'] = masks
        target['image_id'] = image_id
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.ids)

In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.ToTensor()
])

# Paths to images and COCO annotations
images_dir = LOG_LABEL_DATASET_FOLDER
train_json = os.path.join(COCO_ANNOTATIONS_DIR, 'train.json')
val_json = os.path.join(COCO_ANNOTATIONS_DIR, 'val.json')

# Dataset instances
train_dataset = COCODataset(root=images_dir, annotation=train_json, transforms=transform)
val_dataset = COCODataset(root=images_dir, annotation=val_json, transforms=transform)

# Collate Function
def collate_fn(batch):
    return tuple(zip(*batch))

# DataLoader instances
train_loader = DataLoader(
    train_dataset,
    batch_size=1,       
    shuffle=True,
    num_workers=0,     
    collate_fn=collate_fn  
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,      
    shuffle=False,
    num_workers=0,      
    collate_fn=collate_fn  
)

print("Datasets loaded successfully for Mask R-CNN training.")

In [5]:
# Visualize a sample from the COCO dataset
def visualize_sample(dataset, index=None):
    if index is None:
        index = random.randint(0, len(dataset) - 1)
    
    img, target = dataset[index]
    img = img.permute(1, 2, 0).numpy()
    
    boxes = target['boxes'].numpy()
    labels = target['labels'].numpy()
    masks = target['masks'].numpy()
    
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    
    ax = plt.gca()
    
    for box in boxes:
        xmin, ymin, xmax, ymax = box
        rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red', linewidth=2)
        ax.add_patch(rect)
    
    plt.axis('off')
    plt.show()

# Visualize a random training sample
visualize_sample(train_dataset)
visualize_sample(val_dataset)

NameError: name 'train_dataset' is not defined

In [4]:
transform = transforms.Compose([
    transforms.ToTensor()
])
images_dir = MY_LOG_LABEL_DATASET_FOLDER
dataset_json = os.path.join(MY_COCO_ANNOTATIONS_DIR, 'dataset.json')
dataset = COCODataset(root=images_dir, annotation=dataset_json, transforms=transform)
visualize_sample(dataset, index=0)

NameError: name 'COCODataset' is not defined